## LangChain Expression Language (LCEL)

LCEL (LangChain Expression Language) est un langage déclaratif et composable conçu pour orchestrer des chaînes de traitement avec des LLM (Large Language Models). Il permet de composer des chaînes d'étapes (prompts, modèles, outils, fonctions, etc.) de manière lisible, modulaire et facilement testable.

### Objectif
Faciliter la création de pipelines LLM complexes en combinant différentes composantes (prompts, modèles, outils, formatteurs) sans écrire trop de logique impérative.

### Principes clés

- Composable : chaque composant peut être combiné avec d'autres via des opérateurs (|, +, etc.).
- Lisible : syntaxe proche d’un pipeline, facile à comprendre.
- Testable : chaque étape peut être testée indépendamment.
- Réutilisable : permet d'encapsuler des chaînes en sous-composants.

### Exécution par séquence

In [ ]:
from langchain_core.runnables import RunnableSequence, RunnableParallel
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableParallel
from langchain_core.tools import tool
import random
load_dotenv()


@tool
def execute_sql(code : str) -> str :
    """
    Cette fonction permet d'executer le code sql passer comme argument
    """
    print("----execution de l'outil en cours....")
    x = random.randint(0,10)
    cond = x % 2

    if cond == 0:
        return "le code s'est exécuté correctement."
    else :
        return "le code n'est pas correct."


def sql_translator(input, instructions):

    chat_model= ChatOpenAI(model_name="gpt-4o-mini")
    llm_with_tool = chat_model.bind_tools([execute_sql])

    prompt= ChatPromptTemplate.from_messages(
        [
            ("system" , f"{instructions}"),
            ("human" , "{input}")  
        ]
    )

    chain = RunnableSequence(prompt, llm_with_tool)
    call = chain.invoke(input)
    if call.content:
        return call.content
    return call.additional_kwargs


instruction ="""
Tu es un traducteur de texte en sql ou de sql en texte. 
Tu prends les instructions données en languages 
naturel pour les traduit en code sql ou du code sql en language naturel.
renvoie uniquement la traduction et ne fait pas la conversation.

Lorsque l'utilisateur te donne un code sql pour traduire en texte
execute le d'abord pour savoir si le code est correct ou pas en faisant appel
à l'outil "execute_sql".
"""
input = """
SELECT
  e.id,
  e.first_name,
  e.last_name,
  qs.q4_2022-qs.q3_2022 AS sales_change
FROM employees e
JOIN quarterly_sales qs
ON e.id = qs.employee_id
WHERE qs.q4_2022-qs.q3_2022 < 0;
"""
print(sql_translator(input, instruction))

### Exécution parallèle

In [ ]:
instruction2= """
Tu es Saray, une agente commmerciale de chez Nodes United Bank.
Tu es responsable des offres commerciales. Lorsque le client te donne son nom complet,
verifie s'il est éligible à la promotion en utilisant l'outil searchEliblige
"""
instruction = """
Tu es Otniel, un agent du service de dette de chez Nodes United Bank.
Tu es responsable des crédit. Lorsque le client te donne son nom complet,
verifie s'il doit la banque en utilisant l'outil searchDettes
"""

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableParallel
from langchain_core.tools import tool
import pandas as pd

@tool
def searchEliblige(nom : str,prenom : str) -> dict | str:

    """
    Cette fonction permet de rechercher les clients qui sont éligibles aux offres promotionnelles.
    """
    df=pd.read_csv("/workspaces/base64-ai/data/clients_eligibles.csv")
    cond = df['nom'] == nom.lower().capitalize()
    cond2 = df['prénom'] == prenom.lower().capitalize()

    if df.loc[cond].empty==False and df.loc[cond2].empty==False :
        return {"fullname" : f"{prenom} {nom}", "L'indentifiant" : f"{df.loc[cond]['client_id'].values[0]}","Eligible  Promo" :True}
    return "le client n'existe pas"

@tool
def searchDettes(nom : str,prenom : str) -> dict | str:
    """
    Cette fonction permet de rechercher les clients qui sont débiteurs.
    """
    df=pd.read_csv("/workspaces/base64-ai/data/clients_debiteurs.csv")
    cond = df['nom'] == nom.lower().capitalize()
    cond2 = df['prénom'] == prenom.lower().capitalize()

    if df.loc[cond].empty==False and df.loc[cond2].empty==False :
        return {"fullname" : f"{prenom} {nom}", "L'indentifiant" : f"{df.loc[cond]['client_id'].values[0]}","dette" :True}
    return "le client n'existe pas"



# Premier niveau d'abstraction qui est le model
llm = init_chat_model(model="gpt-4o-mini", model_provider="openai")
llm_dette = llm.bind_tools([searchDettes])
llm_com = llm.bind_tools([searchEliblige])


# Deuxieme niveau d'abstraction qui represente les messages
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"{instruction}"),
        ("human", "{content}")
    ]
)

agent_dette = prompt | llm_dette

prompt2= ChatPromptTemplate.from_messages(
    [
        ("system", f"{instruction2}"),
        ("human" , "{input}")
    ]
)
agent_commercial= RunnableSequence(prompt,llm_com)


team = RunnableParallel({
    "agent_dette":agent_dette,
    "agent_com":agent_commercial
})


call = team.invoke("Nkodia Alice")
print(call["agent_dette"])
print('-----------------')
print(call["agent_com"])

content='' additional_kwargs={'tool_calls': [{'id': 'call_Qxhv0013oBTc11jHMcvqF0fQ', 'function': {'arguments': '{"nom":"Nkodia","prenom":"Alice"}', 'name': 'searchDettes'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 67, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BaOV9B5t7ArNLHvkKvRIq52OczY9B', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--0b41b100-195c-46f7-8443-7d829852f380-0' tool_calls=[{'name': 'searchDettes', 'args': {'nom': 'Nkodia', 'prenom': 'Alice'}, 'id': 'call_Qxhv0013oBTc11jHMcvqF0fQ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 67, 'output_tokens': 20, 'total_tokens': 87, 'input_t

In [ ]:
from langgraph.prebuilt import create_react_agent

x = create_react_agent(model='gpt-4o-mini', tools=[searchDettes], prompt=prompt)

ValueError: Message dict must contain 'role' and 'content' keys, got {'content': 'Nkodia Alice'}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 